# Menadione Tn-seq notebook

In [ ]:
#download from genewiz sftp
sftp jstien01_student_bbk@gweusftp.azenta.com
lcd /d/in16/u/sj003/men_tnseq
cd 40-842749567
mget *

In [21]:
# files in /d/in16/u/sj003/men_tnseq/fastq

#!ls fastq
!zcat A1_R1_001.fastq.gz | head -5
!zcat A1_R2_001.fastq.gz

gzip: A1_R1_001.fastq.gz: No such file or directory
gzip: A1_R2_001.fastq.gz: No such file or directory


### File checks to make sure downloaded correctly--matching sizes?

In [ ]:
#check length of files to see if reads same in all files?
!cd fastq
!FILES=*.fastq.gz
!for file in $FILES; do wc -l $file; done >> sanity_check.txt
# all 3 files have different line counts, R2 is index. R1 and R3 more similar, different because of line continuation?

# check files downloaded correctly
!for file in $FILES; do md5sum -c $file.md5; done >> md5_check.txt

#all 'OK'

#head
!for file in $FILES; do echo $file; zcat $file | head -10 $file; done


line count was the same for R1 and R2 when we did cattle in vivo

In [8]:
# for A1 sample see how many times header appears in file
!cd fastq
!zgrep -c 'N:0:AACGTGAT' A1_R1_001.fastq.gz
#64796903
!zgrep -c 'N:0:AACGTGAT' A1_R2_001.fastq.gz
#64796903
!zgrep -c 'N:0:AACGTGAT' A1_R3_001.fastq.gz
#64796903

gzip: A1_R1_001.fastq.gz: No such file or directory


ON mac the zgrep command doesn't seem to work (or zcat?) use gunzip?

This indicates same number of reads in all files.

Run fastqc and then can double-check read length for all files

In [20]:
!cd fastq
!module load fastqc
!module load multiqc
!FILES=*.fastq.gz
!for f in $FILES; do fastqc ${f} -o fastqc; done
!cd fastqc
!multiqc .


SyntaxError: invalid syntax (<ipython-input-20-a2cea0d7dfa0>, line 2)

In [1]:
#test bash command

pwd
FILES=*fastq.gz
for file in $FILES; do echo $file; done

SyntaxError: invalid syntax (<ipython-input-1-03ca3f647ae6>, line 5)

These look fine--reads of 151bp for R1 and R3 (corresponding to Read 1 and Read 3) and i7 read (R2) is 8bp which represents the barcode.

Next step is to iterate through reads in R1 and R2 to 'demultiplex' by combining to one 'template' the reads that have the same barcode in R2.


In [6]:
!zcat fastq/A1_R2_001.fastq.gz | head -20

@A01968:63:H77VYDSX5:4:1101:28212:1031 2:N:0:AACGTGAT
CGCAGTAT
+
FFFFFF:F
@A01968:63:H77VYDSX5:4:1101:29369:1031 2:N:0:AACGTGAT
GAGCACCT
+
FF:FFFFF
@A01968:63:H77VYDSX5:4:1101:32027:1031 2:N:0:AACGTGAT
CATACCAA
+
FFFFFFFF
@A01968:63:H77VYDSX5:4:1101:3341:1047 2:N:0:AACGTGAT
GTACATGC
+
:FFFF::F
@A01968:63:H77VYDSX5:4:1101:9073:1047 2:N:0:AACGTGAT
GCCGCCCC
+
FFFFFFFF

gzip: stdout: Broken pipe


Line 1 is the read identifier with the barcode

In [51]:
!zcat fastq/A1_R1_001.fastq.gz | head -20
!zcat fastq/A1_R1_001.fastq.gz | head -1000 > test_1000_R1.txt

zcat: can't stat: fastq/A1_R1_001.fastq.gz (fastq/A1_R1_001.fastq.gz.Z): No such file or directory


Testing on laptop need to install gnused before can use sed

In [41]:
# find all unique reads in R2 line 2 of each read
#!head -1000 data/fastq/A1_R2_001.fastq > test_1000_R2.txt
#!brew install gnu-sed
#!sed -n '1~2p' test_1000_R2.txt | sort | uniq -c | sort -nr
# need gsed installed for this
#!awk 'NR % 2 == 0' data/fastq/test_1000_R2.txt | sort | uniq -c | sort -nr
# print every 4 lines starting from line 2
!awk 'NR % 4 == 2' data/fastq/test_1000_R2.txt > test.txt
cat test.txt


In [44]:
!sort test.txt | uniq -c | sort -nr

  12 GGGGGGGG
   3 ACAAGAAT
   2 GCCGCCCT
   2 CGGCGTCA
   2 ATTGAAGT
   2 ATATACAA
   2 ACTATCGT
   2 AATTAACT
   2 AAGGTTAA
   1 TTTTGAGG
   1 TTTTACCT
   1 TTTAGTCA
   1 TTGCACGA
   1 TTCGTGCA
   1 TTCGTACT
   1 TTCCCAGT
   1 TTCATAAG
   1 TTCACGTC
   1 TTAGTAAC
   1 TTAGACAT
   1 TTAAAACC
   1 TGTTGTGA
   1 TGTTGTCT
   1 TGTCCATC
   1 TGGTTCAA
   1 TGCACTTT
   1 TGCAACGG
   1 TGATCTTA
   1 TGAGCACA
   1 TGACATGA
   1 TGACATCT
   1 TGACATAT
   1 TCTTGTCA
   1 TCTGCCTC
   1 TCTACTTC
   1 TCTAACGC
   1 TCGTTCAG
   1 TCGTCTTA
   1 TCGTCCAG
   1 TCATTGAA
   1 TCATTCAT
   1 TCATGTTA
   1 TCACATGC
   1 TCACAAGT
   1 TCAATTAC
   1 TCAACAGC
   1 TATTAATT
   1 TATTAAGC
   1 TATGATCT
   1 TAGTCAAG
   1 TAGTATTT
   1 TAGCTGAT
   1 TAGCATAA
   1 TAGACTCA
   1 TACTTGAC
   1 TACTATTC
   1 TACGATAC
   1 TAATCATC
   1 TAAGACCC
   1 TAAAATCA
   1 GTTCTGCT
   1 GTGCCGAG
   1 GTGACCCA
   1 GTGACACG
   1 GTCTAATT
   1 GTCCACAT
   1 GTCCAACT
   1 GTAGCATG
   1 GTACCGAT
   1 GTACATGC
   1 GTAATGCA
   1 G

This shows how many duplicates of certain barcodes. need to match barcodes with read name to eliminate ones that have repeats

In [47]:
#do again with whole file (takes a really long time)
!awk 'NR % 4 == 2' data/fastq/A1_R2_001.fastq | sort | uniq -c | sort -nr > count_barcodes.txt

# top counts
1312199 GGGGGGGG
16805 CAAAACGA
16371 GCAACATT
14835 AATACAAC
14507 CCAAAGAC
14483 AATATAAC
14427 TAACAACA
14120 AAAACATA
13989 TAAAACTA
13544 CCGAAACA

Find distribution of these reads

Which reads are associated with these really big duplications?

In [50]:
# thiss just puts pattern in file--not whole read
!grep 'GGGGGGGG' data/fastq/A1_R1_001.fastq > gggg_dupl.fastq

paste two files together so barcode is added onto read header?

In [ ]:
#bash solution
!paste -d " " file1.txt file2.txt

Make a new fastq file 

In [127]:
#python
seq_list = []
with open ('test_1000_R1.txt', 'r') as f1, open('test_1000_R2.txt', 'r') as f2:
    #line1_f1 = f1.readline()
    #for line in f1.read().split("\n")[0]:
    #    print(line.rstrip())
    # header and sequence for R1 reads
    for lineno, line in enumerate(f1):  
        if lineno % 4 == 0:
            f1_head = line.rstrip()
        # second line and every 4  (sequence)  
        if lineno %4 == 1:
            #print(line)
            f1_seq = line
        # spacer line
        if lineno %4 ==2:
            f1_spacer = line
        #quality line
        if lineno %4 ==3:
            f1_quality = line

        
    #create header and barcode for index reads
    for lineno, line in enumerate(f2):
        if lineno % 4 == 0:
            f2_head = line.rstrip()
            
        if lineno % 4 == 1:
            #print(line)
            f2_barcode = line.rstrip()
        # add barcode to end of header in read1 file
        new_head = f1_head + " BC:" + f2_barcode

        list_entry = new_head + "\n" + f1_seq + f1_spacer + f1_quality
    
        seq_list.append(list_entry)

# write new file with barcodes
with open('new_barcode_file_1000.txt', 'w') as outfile:
    outfile.writelines(seq_list)
outfile.close()

Looking at the sequences for the R1 reads that have the GGGGGGGG 'random barcode' in the first 1000 reads, these sequences all have huge runs of G's?

"For Illumina NextSeq/NovaSeq data, polyG can happen in read tails since G means no signal in the Illumina two-color systems. fastp can detect the polyG in read tails and trim them. This feature is enabled for NextSeq/NovaSeq data by default, and you can specify -g or --trim_poly_g to enable it for any data, or specify -G or --disable_trim_poly_g to disable it. NextSeq/NovaSeq data is detected by the machine ID in the FASTQ records. " 
Need to map reads first because will exclude artefactual PCR amplicons that have identical P7 as well as same genomic insertion site. (from fastp documentation, https://github.com/OpenGene/fastp)

But maybe need to add the barcode to the header of each read first. Adapt above to make python script that can work on entire files of reads and make new files

In [1]:

def add_barcode(file1, file2):
    """
    A function to extract the barcode from the associated p7 index read to the header of each read in the fastq file

    Input               file1                       fastq file for sequence reads (R1 or R3)
                        file2                       fastq file for i7 index reads (R2)
    Output              barcoded<sample>.fastq      new fastq file which has barcode added to header   

    """
    import re
    import os.path
    import sys

    # identify sample name
    filename = str(file1)
    bn_sample = os.path.basename(file1)
    sample = re.sub(".fastq", "", bn_sample)
    print(sample)

    seq_list = []
    with open (file1, 'r') as f1, open(file2, 'r') as f2:
        #count lines in R1 and R2 and proceed if equal
       # if sum(1 for _ in f1) != sum(1 for _ in f2):
        #    sys.exit("Number of reads do not match")
            
    # header and sequence for R1 reads
        for index, (line1, line2) in enumerate(zip(f1, f2)):
            if index % 4 == 0:
                f1_head = line1.rstrip()
                f2_head = line2.rstrip()
            # second line and every 4  (sequence)  
            if index %4 == 1:
                f1_seq = line1.rstrip()
                #create header and barcode for index reads
                #>A01968:63:H77VYDSX5:4:1101:25455:1423 1:N:0:AACGTGAT BC:GGGGGGGG
                f2_barcode = line2.rstrip()
                # add barcode to end of header in read1 file
                new_head = f1_head + "_BC:" + f2_barcode
                #replace whitespace
                new_head = new_head.replace(" ", "_")
                
            # don't need these if do after trimming
            #quality line
            if index %4 ==3:
                f1_quality = line1.rstrip()
                list_entry = new_head + "\n" + f1_seq + "\n" + "+" + "\n" + f1_quality + "\n"
                seq_list.append(list_entry)
    print(len(seq_list))
    # write new file with barcodes
    new_filename = "barcode_" + sample + ".fastq"
    with open(new_filename, 'w') as outfile:
        outfile.writelines(seq_list)
    outfile.close()


In [2]:
add_barcode("test_5000_R1.fastq", "test_5000_R2.fastq")

test_5000_R1
1250


In [97]:
add_barcode("data/fastq/A1_R1_001.fastq", "data/fastq/A1_R2_001.fastq")

A1_R1_001


In [183]:
# can also apply to read2, but no point as won't be using for mapping
#add_barcode("data/fastq/A1_R3_001.fastq", "data/fastq/A1_R2_001.fastq")

A1_R3_001
64796903
barcode_A1_R3_001.fastq


Map small file to see if header stays connected to read (installed new bwa-mem2)


In [ ]:
# sample first 10000 bases of barcoded fastq
!head -10000 barcode_A1_R1_001.fastq > new_barcode_file_A1_10000.fastq 


In [ ]:
!conda activate tnseq
# have to index file first for bwa-mem2
!bwa-mem2 index ref_seqs/Mbovis_AF2122-97.fasta
!bwa-mem2 mem -C ref_seqs/Mbovis_AF2122-97.fasta barcode_test_5000_R1.fastq > test_map_5000.sam
# still getting error: ERROR! Unable to open the file: ref_seqs/Mbovis_AF212297.fasta.bwt.2bit.64
# there were multiple fasta files, deleted all others and manually coppied and pasted text from ref file. changed to Mbovis_AF2122-97.fasta




barcode doesn't appear in sam header automatically, -C to append FASTA/FASTQ comment to SAM output (appends to end of file--column 12 in sam file) (-V            output the reference FASTA header in the XR tag--gives blank file)

Column 4 in sam header is meant to be the start mapping position (should be insert position). in test file, all have start position as 0--probably didn't map?

In [13]:
#parse sam file for start and barcode

def parse_samfile(samfile):
    import pandas as pd
    """
    find molecular barcode and start of mapping in each read
    """

    sam_list = []
    barcodes = []
    id_dups = []
    data = pd.read_csv(samfile, sep="\t", skiprows=2, header=None)
    #print(data.head())
    ids = data[0]
    starts = data[3]
    # sometimes dividing up final lines? 13 for 1000 or 15 for 100000
    barcodeLines = list(data[15])
    #extract molecular barcode
    for line in barcodeLines:
        barcodes.append(line.split("BC:",1)[1])
    # remove non-unique rows?
    #dup_tup = pd.DataFrame(ids)
    dup_tup = pd.DataFrame(starts)
    dup_tup.columns=["starts"]
    dup_tup['barcodes'] = barcodes
    # get count of duplicates for each unique row
    sorted_dup_tup = dup_tup.groupby(dup_tup.columns.tolist(), as_index=False).size()
    
    return sorted_dup_tup
    
    
p = parse_samfile("test_map_10000.sam")
print(p.head())

    starts  barcodes  size
0  2736492  AAAAAAAT     1
1  2736492  AAAAAAGG     1
2  2736492  AAAAACAG     1
3  2736492  AAAAACAT     1
4  2736492  AAAAACCT     1


column 3 (0 indexed) is start, column 13/15 has both barcodes, column 9 is sequence. No duplicate start/barcode combos apparent in the 10000bp test file.

Better approach may be to process reads and find duplicates using Picard find duplicates tool (Mark Duplicates)

https://gatk.broadinstitute.org/hc/en-us/articles/360037051452-EstimateLibraryComplexity-Picard-

can use argument --BARCODE_TAG 'BC' to identify barcode
--READ_NAME_REGEX null if duplicate sets are very high and we are not trying to establish library complexity

Reads must be mapped and sorted

In [ ]:
!conda install -c bioconda picard
!java -jar picard.jar MarkDuplicates \
      I=input.bam \
      O=marked_duplicates.bam \
      M=marked_dup_metrics.txt

First step before mapping is to remove the transposon sequence from each read
## From TRANSIT/TPP docs:

1. Convert .fastq files to .fasta format (.reads).
    https://bioinformaticsworkbook.org/dataWrangling/fastaq-manipulations/converting-fastq-format-to-fasta.html#gsc.tab=0
    This would only keep header and sequence.

(creates .reads file using 'fastq2reads')

In [176]:
gsed -n '1~4s/^@/>/p;2~4p' new_barcode_file_1000.fastq > new_barcode_file_1000.fasta


2. Identify reads with the transposon prefix in R1 . The sequence searched for is ACTTATCAGCCAACCTGTTA (or TAAGAGACAG for Tn5), which must start between cycles 5 and 10 (inclusive). (Note that this ends in the canonical terminus of the Himar1 transposon, TGTTA.) The “staggered” position of this sequence is due to insertion a few nucleotides of variable length in the primers used in the Tn-Seq sample prep protocol (e.g. 4 variants of Sol_AP1_57, etc.). The number of msmatchen searching reads for the transposon sequence pattern can be adjusted as an option in the interface; the default is 1.

    Use fastp for this? First try with TPP?


In [ ]:
#(thoth)
#!tpp -bwa /s/software/bwa/bwa/bwa -ref $my_path/refseqs/Mtb/Mtb_H37Rv.fasta \
#    -reads1 $my_path/ncbi/files/dejesus/SRR4113428_1.fastq \
#        -reads2 $my_path/ncbi/files/dejesus/SRR4113428_2.fastq \
#            -output $my_path/dejesus_mtb/tpp_results/SRR4113428

# paired-end (laptop)
# this will look for barcode which isnt' there, but maybe useful to trim read 2 and extract genomic sequence?
!tpp -bwa ~/anaconda3/envs/tnseq/bin/bwa-mem2 -ref ref_seqs/Mbovis_AF2122-97.fasta \
    -reads1 barcode_A1_R1_001.fastq -reads2 data/fastq/A1_R3_001.fastq \
        -output tpp_test_A1_paired

# Error: unexpected format of headers in .fastq files
#Assume this is due to adding barcode to header for read1



Fastp can use index reads for UMI processing--can extract from index reads and append to first part of read names so it will be in sam/bam files, can specify prefix 'UMI'

In [181]:
!fastp -i data/fastq/A1_R1_001.fastq -I data/fastq/A1_R3_001.fastq -o data/fastp_out/A1_R1_001_trimmed.fastq -O data/fastp_out/A1_R3_001_trimmed.fastq \
 -U --umi_loc=index1 --umi_prefix=UMI --unpaired1 -l=20 --adapter_fasta adapter_file.txt

#It reads from index that is already in multiplexed header--not from index file, so not useful unless specify P7 reads as R2 and it can get umi from read2
#--unpaired1 means will save unpaired read 1s, -l 20 means will require reads to be at least 20 bp after trimming

zsh:1: command not found: fastp


Added an 'adapter file' to use with fastp: 'adapter_file.txt' which includes the transposon sequence pattern for R1 and R2.

This is leading to very short R1 sequences which actually match the transposon sequence (no genomic?) Are R1 and R3 reversed and R3 is really read1?

Do tests with shorter number of reads (100000)

Use R2 index reads as read2?

#temporarily disable adapter trimming 

In [ ]:
#fastp -i data/fastq/A1_R1_001.fastq -I data/fastq/A1_R2_001.fastq -o data/fastp_out/A1_R1_001_trimmed.fastq -O data/fastp_out/A1_R3_001_trimmed.fastq \
 #-U --umi_loc=read2 --umi_prefix=UMI --umi_len=8 --unpaired1 -l 20 --adapter_fasta adapter_file.txt --reads_to_process=100000

fastp -i data/fastq/A1_R1_001.fastq -I data/fastp_out/A1_R2_001_trimmed.fastq \ 
-U --umi_loc=read2 --umi_prefix=UMI --umi_len=8 --reads_to_process=100000 -o data/fastp_out/A1_R1_001_trimmed.fastq -O data/fastp_out/A1_R3_001_trimmed.fastq
 
 #this doesn't work--needs to have properly paired ends

This is new heading that fastp makes that includes UMI: @A01968:63:H77VYDSX5:4:1101:32027:1031:UMI_AACGTGAT 1:N:0:AACGTGAT
I imagine this will work for TPP and still have heading included in mapped reads if I just move it to end of first field instead of randomly adding to end. MIght be easier to filter with picard then. but not sure it matters if I don't do tpp since I can parse from sam files myself--it is only using P5 index from the header of the reads--NOT UMI barcode from p7 index files

In [ ]:
#unpaired fastp for adapter trimming
!fastp -i new_barcode_file_A1_10000.fastq -o data/fastp_out/test_A1_R1.fastq --adapter_fasta adapter_file.txt

This is just leaving super short reads like before (15 bp long--also still part of primer, no genomic sequence left). Maybe because I have both ends of inverted repeat sequence in there

@A01641:207:HNJLJDSX3:1:2678:31331:37059 1:N:0:AACGTGAT BC:CGCAGTAT
GTCTAGAGACCGGGG
+
F::FF:FFFFFFFFF

In [ ]:
# without adapter fasta file--using transposon seq for R1 only
!fastp -i data/fastq/A1_R1_001.fastq -o data/fastp_out/test_repeat_A1_R1.fastq --adapter_sequence=ACTTATCAGCCAACCTGTTA --reads_to_process=100000
#results in mostly 15 bp identical sequences
#reads with adapter trimmed: 98991 (trims nearly all)

#try with R3
!fastp -i data/fastq/A1_R3_001.fastq -o data/fastp_out/test_repeat_A1_R3.fastq --adapter_sequence=ACTTATCAGCCAACCTGTTA --reads_to_process=100000
#reads with adapter trimmed: 17

#using Read 2 transposon sequence:
!fastp -i data/fastq/A1_R3_001.fastq -o data/fastp_out/test_repeat_A1_R3.fastq --adapter_sequence=TGGTCGTGGTAT --reads_to_process=100000
#bases trimmed due to adapters: 7559

# using rc of read 2 transposon sequence:
!fastp -i data/fastq/A1_R3_001.fastq -o data/fastp_out/test_repeat_A1_R3.fastq --adapter_sequence=TACCACGACCA --reads_to_process=100000


# using automatic detection
!fastp -i data/fastq/A1_R1_001.fastq -o data/fastp_out/test_automatic_A1_R1.fastq --reads_to_process=100000
#finds truseq adapter read1, but not many are actually trimmed

First trim with sequencing adapters. Then do again in separate step (cutadapt) to cut off transposon prefix

added barcode info to both paired-end reads (R1 and R3). After mapping will try to remove duplicates and make wigs again? Will have to put into header of both reads as TPP needs these to match.

Paired end takes hours for single sample. Not going to be useful as R2 contains empty barcodes. Use single-end to see if get anything useful. otherwise do manually and compare with trimmed files 

In [ ]:
#tpp on paired ends with matched headers. doesn't map with no barcodes on 2nd read (set for paired end#)
!tpp -bwa ~/anaconda3/envs/tnseq/bin/bwa-mem2 -ref ref_seqs/Mbovis_AF2122-97.fasta \
    -reads1 barcode_A1_R1_001.fastq -reads2 barcode_A1_R3_001.fastq -output tpp_test_A1_paired

# try for single end 
#!tpp -bwa ~/anaconda3/envs/tnseq/bin/bwa-mem2 -ref ref_seqs/Mbovis_AF2122-97.fasta \
#    -reads1 barcode_A1_R1_001.fastq -output tpp_test_A1_single

#on thoth
!tpp -bwa /s/software/bwa/bwa/bwa -ref /d/in16/u/sj003/refseqs/mbovis/Mbovis_AF2122_97.fasta \
    -reads1 fastq/A1_R1_001.fastq.gz -output tpp/tpp_test_A1_single
#TypeError: sequence item 1: expected str instance, bytes found
#gunzipped file and tried again with unzipped file--seemed to work, but want to use barcoded fastq instead so aborted
#!gunzip fastq/A1_R1_001.fastq.gz
#!tpp -bwa /s/software/bwa/bwa/bwa -ref /d/in16/u/sj003/refseqs/mbovis/Mbovis_AF2122_97.fasta \
#    -reads1 fastq/A1_R1_001.fastq -output tpp/tpp_test_A1_single

#zip file on laptop, send to thoth, try again
!gzip barcode_A1_R1_001.fastq 
!scp barcode_A1_R1_001.fastq.gz sj003@ssh.cryst.bbk.ac.uk:/d/in16/u/sj003/men_tnseq/fastq/
!gunzip fastq/barcode_A1_R1_001.fastq.gz
!tpp -bwa /s/software/bwa/bwa/bwa -ref /d/in16/u/sj003/refseqs/mbovis/Mbovis_AF2122_97.fasta \
    -reads1 fastq/barcode_A1_R1_001.fastq -output tpp/tpp_test_A1_single

#barcoded reads were wrong--all had same sequence information (bug in program)




3. Extract genomic part of read 1. This is the suffix following the transposon sequence pattern above. However, for reads coming from fragments shorter than the read length, the adapter might appear at the other end of R1, TACCACGACCA. If so, the adapter suffix is stripped off. (These are referred to as “truncated” reads, but they can still be mapped into the genome just fine by BWA.) The length of the genomic part must be at least 20 bp.

4. Extract barcodes from read 2. Read 2 is searched for GATGGCCGGTGGATTTGTGnnnnnnnnnnTGGTCGTGGTAT”. The length of the barcode is typically 10 bp, but can be varaible, and must be between 5-15 bp.
    -can tpp skip this step (it takes absolutely forever)?
    -can this be done manually and fed in? or can I use header from .sam files in step 8 instead of barcode

5. Extract genomic portions of read 2. This is the part following TGGTCGTGGTAT…. It is often the whole suffix of the read. However, if the read comes from a short DNA fragment that is shorter than the read length, the adapter on the other end might appear, in which case it is stripped off and the nucleotides in the middle representing the genomic insert, TGGTCGTGGTATxxxxxxxTAACAGGTTGGCTGATAAG. The insert must be at least 20 bp long (inserts shorter than this are discarded, as they might map to spurious locations in the genome).
    -can this work in paired end without barcodes? 

6. Map genomic parts of R1 and R2 into the genome using BWA. Mismatches are allowed, but indels are ignored. No trimming is performed. BWA is run in ‘sampe’ mode (treating reads as pairs). Both reads of a pair must map (on opposite strands) to be counted.
    

7. Count the reads mapping to each TA site in the reference genome (or all sites for Tn5).

8. Reduce raw read counts to unique template counts. Group reads by barcode AND mapping location of read 2 (aka fragment “endpoints”).
    -i can do this with mapped reads


Read 1 is successfully trimmed using himar1 sequence. Read2 isn't trimmed (can't find himar1 sequence). Can use these files for ds analysis? (_paired.trimmed1). This is final fastq for read1 for tpp. _paired.genomic2 is final file for read 2.

I should maybe map using .trimmed1 and .trimmed2? or run tpp in single-end mode only

Mendum, smith, butler and that group only gets single-end reads from sequencing. OK to ignore read 2

Alternate (non-tpp) strategy
1. add barcodes to read 1 from index reads 
2. fastp for illumina trimming and QC (.fastq, filtered and trimmed)
3. then map with bwa-mem2 (.sam/.bam)
4. then filter reads for ...TGGTA in first 32 bp of read and start/barcode (filtered and mapped)
5. then count template reads mapping to each TA site (generate .wig files)
6. use transit for essentiality/resampling

## pipeline

1. add barcodes to Read1 from index reads


In [ ]:
add_barcode(READ1, INDEX)


2. fastp for qc and illumina adapter trimming

I believe the reads have already been trimmed by genewiz when multiplexed as fastp doesnt detect many of the sequencing adapters. Could skip trimming, but useful for shortening test file and also qc and catching any untrimmed adapters.


In [ ]:
#automatic adapter trimming
#!fastp -i barcode_A1_R1_001.fastq.gz -o trimmed/test_A1_R1_trimmed.fastq --reads_to_process=10000
# this trims all reads and nothing left!
#Detecting adapter sequence for read1...
#GTCTAGAGACCGGGGACTTATCAGCCAACCTGTTAACCGAGGCATCCCAGAGCTTGCCGG

#this is trasposon sequence and also genomic dna. skip trimming and map without? more aggressive trimming using barcode processed file

#disable adapter trimming or indicate only truseq adapter

!fastp -a GATCGGAAGAGCACACGTCTGAACTCCAGTCAC -i barcode_A1_R1_001.fastq -o trimmed/test_A1_R1_trimmed.fastq --reads_to_process=10000

#on thoth
!fastp -a GATCGGAAGAGCACACGTCTGAACTCCAGTCAC -i fastq/barcode_A1_R1_001.fastq -o trimmed/barcode_A1_R1_001.fastq

3. Map read 1 with bwa-mem2 and sort
    -can use TPP single end for this?

    Use snakemake mapping/sorting script

In [ ]:
# !conda activate tnseq
# # have to index file first for bwa-mem2
# !bwa-mem2 index ref_seqs/Mbovis_AF2122-97.fasta
# !bwa-mem2 -t 3 -C ref_seqs/Mbovis_AF2122-97.fasta data/fastp_out/A1_R1_trimmed.fastq > mapped_reads/A1_R1.sam
# # bwa samse?
# #!samtools view -Sb mapped_reads/A1_R1.bam
# !samtools sort -T -o sorted_reads/A1_R1_sorted.sam -O sam mapped_reads/A1_R1.sam 


# with snakemake (maps, sorts, indexes and creates flagstats report)
#make config.yaml file


!cd ~/tn_seq/menadione_tnseq/
!conda activate snakemake
!snakemake -np -s ~/snakemake/tnseq/snakefile.smk
!snakemake --cores 2 -s ~/snakemake/tnseq/snakefile.smk
#!snakemake -np -s $my_path/snakemake/tnseq/snakefile.smk
#!nohup snakemake --cores 8 -s $my_path/snakemake/map_bwa/pe/snakefile.smk > nohup_map.out 2>&1 &

samtools doesn't recognise the .sam format made by bwa in the -C mode includes quality scores, etc? When transit adds barcodes, it does it to names when making read files, so I've moved barcode to name so I don't have to worry about keeping any fields after the sequence.

More info about -C parameter and formation of sam headers:
"Append append FASTA/Q comment to SAM output. This option can be used to transfer read meta information (e.g. barcode) to the SAM output. Note that the FASTA/Q comment (the string after a space in the header line) must conform the SAM spec (e.g. BC:Z:CGTAC). Malformated comments lead to incorrect SAM output."

except for barcode, all my reads are identical. maybe this is something wrong with the barcoded file--since fastp also removed whole read.

Yes--barcode script was wrong. fixed bug. This works for downstream snakemake trimming/mapping pipeline

Filter out unmapped reads--part of snakemake pipeline. Changed snakemake to include header (@SQ) in output .sam file

4. filter reads for transposon sequence (in first 32 bp?)
 

 

In [17]:
#slice <first 32 bp of sequence line> | grep <transposon tag> > concatenate to .tag_positive file



#filter reads in samfile for transposon tag in sequence
#from tpp (https://github.com/mad-lab/transit/blob/master/src/pytpp/tpp_tools.py)
# find index of H[1..m] in G[1..n] with up to max mismatches
# note: this find first match, not necessarily the best (with min mismatches)

def mmfind1(G,n,H,m,max): # lengths; assume n>m
  """
  A function to find matches of transposon sequence in read
    Input               G                           sequence string
                        n                           length of read sequence
                        H                           pattern string
                        m                           length of pattern
                        max                         maximum mismatches
    Output              i, -1                       start position of match, or -1 for no match
  """

  a = G[:n].find(H[:m])
  if a!=-1: return a # shortcut for perfect matches
  for i in range(0,n-m):  # range of 0 to difference between seq length and len pattern
    cnt = 0
    for k in range(m):
      if G[i+k]!=H[k]: cnt += 1
      if cnt>max: break
    if cnt<=max: return i
  return -1
    
def find_tags(samfile, target_tag, mismatch_max=2):
    
    """
    Find transposon tag in sequence of each read.

      Input           samfile           file of mapped reads in .sam format
                      target_tag        string that matches transposon sequence
                      mismatch_max      number of mismatches allowed
      Output          match_list        list of length of number of reads in .sam file 
                                        with match start position or -1 if no match
    """

    import pandas as pd
    import pickle
    import os

    match_list = []
    # this is giving errors in parsing
    data = pd.read_csv(samfile, sep="\t", comment="@", header=None, usecols=[0,1,2,3,4,5,6,7,8,9,10])
    seqs = data[9]
    #search string for transposon seq with 2 mismatches
    for i in range(0, len(seqs)):
      seq = seqs[i]
      res = mmfind1(seq, len(seq), target_tag, len(target_tag), mismatch_max)
      match_list.append(res)
    #print(len(match_list))
    print("Number of reads with no transposon tag: ", match_list.count(-1))
    print("Number of reads with tag within first 32 bases: ", sum(1 for i in match_list if i >0 and i < 32))
    bn = os.path.basename(samfile)
    file_name = bn + "_tags" + ".pkl"
    with open(file_name, 'wb') as f:
      pickle.dump(match_list, f)
    return match_list
      


In [32]:
find_tags("sorted_reads/test_5000_R1.sam", "ACTTATCAGCCAACCTGTTA", 2)

Number of reads with no transposon tag:  611
Number of reads with tag within first 32 bases:  635


[-1,
 15,
 15,
 -1,
 15,
 -1,
 15,
 15,
 -1,
 15,
 -1,
 14,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 15,
 -1,
 15,
 15,
 -1,
 15,
 15,
 15,
 -1,
 15,
 -1,
 -1,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 -1,
 15,
 15,
 -1,
 15,
 15,
 15,
 15,
 15,
 -1,
 -1,
 15,
 -1,
 -1,
 -1,
 15,
 15,
 -1,
 -1,
 15,
 15,
 15,
 15,
 -1,
 15,
 15,
 -1,
 15,
 -1,
 -1,
 15,
 15,
 -1,
 -1,
 15,
 -1,
 -1,
 15,
 15,
 -1,
 -1,
 -1,
 -1,
 15,
 -1,
 -1,
 15,
 -1,
 15,
 15,
 15,
 15,
 -1,
 -1,
 15,
 15,
 -1,
 15,
 15,
 15,
 14,
 15,
 -1,
 15,
 15,
 -1,
 -1,
 -1,
 -1,
 15,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 15,
 15,
 15,
 15,
 -1,
 -1,
 -1,
 15,
 15,
 15,
 15,
 15,
 -1,
 -1,
 15,
 15,
 -1,
 15,
 -1,
 -1,
 15,
 -1,
 -1,
 15,
 15,
 -1,
 -1,
 -1,
 15,
 -1,
 -1,
 -1,
 -1,
 15,
 -1,
 -1,
 15,
 15,
 15,
 -1,
 15,
 15,
 -1,
 15,
 -1,
 15,
 -1,
 -1,
 -1,
 15,
 15,
 15,
 15,
 -1,
 15,
 15,
 -1,
 15,
 15,
 15,
 -1,
 15,
 -1,
 -1,
 -1,
 -1,
 15,
 -1,
 15,
 -1,
 15,
 15,
 15,
 15,
 15,
 -1,
 15,
 -1,
 -1,
 -1,
 -1,
 15,
 -1,


In [4]:
def find_barcodes(read):
    """
    find molecular barcode and start of mapping in each read
    """
    #extract molecular barcode from a single read and return tuple
    
    read_name = read.split()[0]
    barcode = read_name.split("BC:",1)[1]
    read_start = read.split()[3]
    bar_start = (barcode, read_start) 
    return bar_start



In [5]:



def filter_mapped_reads(sam_file, tag="ACTTATCAGCCAACCTGTTA", mis_max=2, position_pkl=[]):
  """ 
  """
  import sys
  import pickle
  import os
  import pandas as pd
  from operator import itemgetter
  if position_pkl == []:
    #create pos_list
    pos_list = find_tags(sam_file, tag, mis_max)
  else:
    with open(position_pkl, 'rb') as f:
      pos_list = pickle.load(f)
  #read sam_file and sort lines between header and reads
  header = []
  reads  = []
  barcode_list = []
  good_reads = []
  notags_reads = []
  bad_reads = []
  with open(sam_file, 'r') as f:
    for line in f:
      line = line.strip()
      if line[0] == "@":
        header.append(line)
      else:
        reads.append(line)
  #compare number of reads in sam file to positions in pos_list
  if len(reads)==len(pos_list):
    for i in range(0, len(pos_list)):
      if pos_list[i] != -1:
        #find barcode/start combo
        bc_start = find_barcodes(reads[i])
        #add to list 
        barcode_list.append(bc_start) #list will essentially be sorted by read start because reads are sorted?
        barcode_list.sort(key=itemgetter(0))  #maybe this will speed up search by barcode?
        # if hasn't been added before, add read to good_reads
        if barcode_list.count(bc_start) < 2:
          good_reads.append(reads[i])
        else:
          bad_reads.append(reads[i])
      else:
        notags_reads.append(reads[i])
    print("number of good reads (with tag): ", len(good_reads))
    print("number of bad reads (with no tag): ", len(notags_reads))
    print("Number of reads with duplicate barcode/starts: ", len(bad_reads))
    bn = os.path.basename(sam_file)
    outfile = "tag_filtered_" + bn
    with open(outfile, 'w') as f:
      for line in header:
        f.write(f"{line}\n")
      for line in good_reads:
        f.write(f"{line}\n")
  else:
    sys.exit("reads file length doesn't match position file")
  
  return barcode_list

In [70]:
filter_mapped_reads("sorted_reads/test_5000_R1.sam")

Number of reads with no transposon tag:  611
Number of reads with tag within first 32 bases:  635
number of good reads (with tag):  603
number of bad reads (with no tag):  611
Number of reads with duplicate barcode/starts:  32


[('AAAAACAG', '274566'),
 ('AAAAACGC', '3897925'),
 ('AAAACTAG', '224546'),
 ('AAAAGTCT', '3648873'),
 ('AAAATGTA', '1384138'),
 ('AAAATTAA', '3373285'),
 ('AAACAAAG', '1165185'),
 ('AAACAAGT', '475611'),
 ('AAACATTG', '3973761'),
 ('AAACCTTT', '2558063'),
 ('AAACTTCC', '985131'),
 ('AAAGCAAA', '2948421'),
 ('AAATAAAC', '0'),
 ('AAATAGTT', '1616431'),
 ('AAATAGTT', '1616431'),
 ('AAATCGCA', '2075169'),
 ('AAATGGAA', '3140968'),
 ('AACAACCA', '2773084'),
 ('AACAACCT', '3499990'),
 ('AACAATGC', '3143862'),
 ('AACACACA', '0'),
 ('AACACCTC', '1696057'),
 ('AACATACT', '1349442'),
 ('AACATTAC', '165903'),
 ('AACCAATC', '925154'),
 ('AACCAGAA', '87640'),
 ('AACTAACC', '0'),
 ('AACTATTC', '3859816'),
 ('AACTCCAA', '1775261'),
 ('AACTGTCA', '3905561'),
 ('AAGACATG', '2183427'),
 ('AAGACCAA', '2883723'),
 ('AAGACGCT', '837708'),
 ('AAGACGTC', '812555'),
 ('AAGATCAC', '307998'),
 ('AAGATGTA', '3847260'),
 ('AAGCAATA', '1480841'),
 ('AAGCCAGA', '1973979'),
 ('AAGCGCTT', '99975'),
 ('AAGGAGCA', '31

5. Filter for PCR amplification reads--reduce to single template counts (do at same time as finding transposon tag)

Will I be able to use the start position to do this?

Need to use new program since barcode now in different place.

1) function to identify barcode and start of each read
2) when found good_read --> apply function and compare barcode:start to list
3) if not in list --> append to good_reads, else to bad_reads

Need to parse sam flag--position is strand specific (sam flag is 2nd field (index=1) and 0 is forward, 16 is reverse). Start position is left-most position of alignment, so for forward it is the start, but for reverse it is the end. To get reverse start, have to add to position the length of the alignment. This is given in the CIGAR string (6th field, so index=5). 

reverse strand (samflag=16): 118M33S  so first number up to 'M' is length of match (33S is softclip of transposon seq)
forward strand (samflag=0): 33S118M so number AFTER S. But this wont need to be parsed, as left-most position is correct for forward.


In [105]:
#parse samflag 

def parse_samflag(read):
    """
    Function to determine strand orientation of alignment from read header
    """
    samflag = read.split()[1]
    if samflag=="16":
        strand = "R"
    elif samflag=="0":
        strand = "F"
    else:
        strand = "*"
    return strand

def align_len(read):

    """
    Function to determine read alignment length from start position of reads 
    filtered for transposon tag (should have soft-clipped ~30-33 before gDNA)

    Some cases may exist like 29S89M33S--transposon at both ends? or adapter not trimmed? 
    if forward, can ignore last part of alignment, if reverse, can ignore first part
    
    """
    import re
    cig = read.split()[5]
    cig_list = parse_cigar(cig)
    cig_last = len(cig_list) - 1
    strand = parse_samflag(read)
    if strand == "R":
        # assume alignment without extra bases at 3' end 
        #make sure no soft-clipping at start
        #if 'M' in cig_list[0]:  #this could be 3' end alignment
        if "S" in cig_list[cig_last]:  #check for soft-clipping (tag) at 5' end of read
            match_len = cig_list[cig_last - 1] #aligned part is next part
            #soft_clip = cig_list[cig_last]
            match_len = int(match_len.split("M")[0])
        else:
            match_len = 0
        
    elif strand == "F":
        #for F strand, softclipped bases at 3' end are cigar_list[2] and can be ignored?, if no soft-clipping at 5' end, will be no tag
        match_len = cig_list[1]
        match_len = int(match_len.split("M")[0])
        #soft_clip = cig_list[0]
    else:
        match_len = 0
    return match_len

def rev_start(pos, alignlen):
    """
    Function to find start position for reverse aligned reads
    """
    rev_start = int(pos) + alignlen
    return rev_start

def parse_cigar(cigar):
    """
    Parse cigar string 
    regex: \*|([0-9]+[MIDNSHPX=])+
    """
    import re
    #keep delimiter
    #cig_list = re.split('([0-9]+M|[0-9]+S)', cigar)
    #remove empty strings
    #cig_list = [i for i in cig_list if i]

    #find all number/letter combos in cigar string
    cig_list = re.findall(r'[0-9]+[MIDNSHPX=]+', cigar)
    return cig_list

In [106]:
#look at align lengths that are less than 50, all aligned lengths > 31

from Menadione_tnseq.ipynb:pylance-notebook-cell:Y144sZmlsZQ== import read_samfile


sf = read_samfile("sorted_reads/mapped_test_5000_R1.sam")[1]
for read in sf:
    x = align_len(read)
    print(x)
  
    # x_al = x[0]
    # x_sc = x[1]
    # sc_num = int(x_sc.split("S")[0])
    # if sc_num < 30:
    # #if "S" not in x_sc:
    #     print(x_al, x_sc)
    #     print(read)
    

118
119
118
118
118
109
118
95
118
69
118
79
118
118
118
118
118
121
121
116
119
118
118
118
118
118
118
118
118
118
119
119
72
118
119
118
113
118
114
118
88
118
118
118
118
118
118
118
89
89
118
118
119
118
118
118
119
118
118
118
118
118
118
119
119
85
118
118
87
118
118
118
118
118
118
118
118
118
111
85
118
118
86
94
118
110
118
107
118
119
118
48
93
101
118
87
93
118
119
118
93
118
118
118
118
119
118
118
118
118
118
47
116
116
92
59
34
118
31
118
118
118
118
109
118
118
119
118
118
75
118
101
122
118
118
118
118
91
118
118
118
118
118
118
118
120
118
107
118
118
119
35
118
118
118
118
118
119
119
118
118
118
118
118
118
67
118
118
113
118
118
118
118
118
109
118
118
53
97
118
85
119
105
118
85
118
118
71
77
118
119
118
118
59
118
118
118
118
115
118
118
118
118
77
62
118
37
118
77
118
119
118
118
70
118
118
73
118
118
42
45
118
118
121
118
118
118
54
118
118
80
119
59
112
118
118
113
102
75
118
118
118
118
118
118
118
118
47
118
118
118
118
118
119
114
117
118
118
104
102
52
102

All aligned lengths are greater than 31. All the ones checked appear to have transposon tag in the soft-clipped region. Could it be more efficient to just check this region for transposon tag in order to filter? This is less reliable because instead of sc, sometimes get D or other tags. Need to check later on that these aren't causing problems with align-length

A01968:63:H77VYDSX5:4:1101:3314:1626_1:N:0:AACGTGAT_BC:ATCGTGAT	16	NC_002945.3	3002949	60	78M1D40M33S	*	0	0	GCGCAGGTGTTGAACACCACGACGTCGGCCTCGGAACCGTCGGTCGCCCTCCGGTAGCCGGCCGCTTCCAGCAGACCCCCAGCCGCTCGGAGTCGTGGACGTTCATCTGACAGCCG*TAACAGGTTGGCTGATAAGTCCCCGGTCTCTAGAC	

In this case, on reverse strand, we want the most 5' aligned region to get start, so this will be closest to the soft clip. So we need to indicate the len(cigar_list)-1 to be the align len, NOT the 1st field. Have tested, and it works for this case, but this is only case in test file that has the D

Shorter sc seem to indicated truncated transposon without the -GTTA, but is this reliable for narrowing search region for tags?

In [3]:
def read_samfile(samfile):
    """
    read sam_file and sort lines between header and reads
    """

    header = []
    reads  = []

    with open(samfile, 'r') as f:
        for line in f:
            line = line.strip()
            if line[0] == "@":
                header.append(line)
            else:
                reads.append(line)
    return header, reads

In [107]:
heads = read_samfile("sorted_reads/mapped_test_5000_R1.sam")[1]
for read in heads:
    sf = parse_samflag(read)
    if sf not in ["R", "F"]:
        print(sf)
        print(read)
        al = align_len(read)
        print(al)
# position = heads.split()[3]
#bc_start = find_barcodes(heads)
#print(bc_start)
# rs = rev_start(position, al)
# print(rs)




re-do find_barcodes, find_tags and filter_mapped_reads to incorporate these functions and find correct start positions. Alignment length can be used to show if transposon tag in first part of read--aligned length needs to be >110 for it to be in first/last 40 bases regardless of strand (unlikely to be at end of aligned read)

In [109]:
def mmfind1(G,n,H,m,max): # lengths; assume n>m
  """
  A function to find matches of transposon sequence in read
    Input               G                           sequence string
                        n                           length of read sequence
                        H                           pattern string
                        m                           length of pattern
                        max                         maximum mismatches
    Output              i, -1                       start position of match, or -1 for no match
  """

  a = G[:n].find(H[:m])
  if a!=-1: return a # shortcut for perfect matches
  for i in range(0,n-m):  # range of 0 to difference between seq length and len pattern
    cnt = 0
    for k in range(m):
      if G[i+k]!=H[k]: cnt += 1
      if cnt>max: break
    if cnt<=max: return i
  return -1

def find_barcodes_starts(read):
    """
    Find barcode and read alignment start position
    Returns tuple of barcode and read start pos
    """
    read_name  = read.split()[0]
    barcode    = read_name.split("BC:",1)[1]
    rd_start = find_start(read)
    bar_start = (barcode, rd_start) 
    return bar_start


def find_start(read):
    """
    Parse read for samflag (strand), start of read position and cigar string for alignment length
    Returns aligned start position. Use this on filtered 'good reads' to identify start for quantification
    """
    strand   = parse_samflag(read)
    al_len   = align_len(read)
    left_pos = read.split()[3]
    if al_len > 0 and strand != "*":
      if strand == "F":
        read_start = left_pos
      else:
        read_start = rev_start(left_pos, al_len)
    else:
        read_start = "-"
    return read_start

def check_seq(str):
    """
    Is the target tag made of acceptable nucleotide characters
    """
    import re
    chars = set('ACTG')
    seq = set(str.upper())
    if seq.issubset(chars):
        return True
    else:
        return False

def find_tags2(read, target_tag, max):
    """
    Find transposon tag in sequence of each read.

      Input           read              mapped read
                      target_tag        string that matches transposon sequence
                      max               maximum number of mismatches allowed (default=2)
      Output          match             match of start of tag from left-most start position of read or -1 if no match
    """
    from Bio.Seq import Seq
    
    # find strand of read
    strand = parse_samflag(read)
    seq = read.split()[9]
    if strand == "F":
        #search string for transposon seq with max num mismatches
        match = mmfind1(seq, len(seq), target_tag, len(target_tag), max)
    elif strand == "R":
        # make rc of seq and search for transposonl tag with max num mismatches
        dna     = Seq(seq)
        rc_seq  = str(dna.reverse_complement())
        match = mmfind1(rc_seq, len(rc_seq), target_tag, len(target_tag), max)
    else: 
        match = -1
    
    return match

def filter_mapped_reads2(sam_file, tag="ACTTATCAGCCAACCTGTTA", mismatch_max=2):
  """ 
  Revised filtering function
  """
  import sys
  import os
  import re
  from operator import itemgetter

  #check tag has valid nucleotides
  if check_seq(tag):
     pass
  else:
     sys.exit("Invalid sequence tag")

  #read sam_file and sort lines between header and reads
  data    = read_samfile(sam_file)
  header  = data[0]
  reads   = data[1]

  barcode_list  = []
  good_reads    = []
  notags_reads  = []
  dup_reads     = []
  
  for read in reads:
    ftag = find_tags2(read, tag, mismatch_max)
    #align length (will parse read align len from header)
    al  = align_len(read)

    if ftag != -1 and al > 0:   #(transposon tag in first 40, but this will discard shorter reads that have tag and have mapped, could make 0)
      #find barcode/start combo
      bc_start = find_barcodes_starts(read)
      barcode_list.append(bc_start) 
      barcode_list.sort(key=itemgetter(0))  #maybe this will speed up search by barcode?
     
      # if hasn't been added before, add read to good_reads
      if barcode_list.count(bc_start) < 2:
        good_reads.append(read)
      else:
        dup_reads.append(read)

    else:
      notags_reads.append(read)
  #save barcode_list as pickle object to use in quantifying reads at ta sites? temporary until read processed?  
  print("Total number of mapped reads: ", len(reads))
  print("number of good reads (with tag): ", len(good_reads))
  print("number of bad reads (with no tag): ", len(notags_reads))
  print("Number of reads with duplicate barcode/starts: ", len(dup_reads))
  bn = os.path.basename(sam_file)
  outfile = "tag_filtered_" + bn
  with open(outfile, 'w') as f:
    for line in header:
      f.write(f"{line}\n")
    for line in good_reads:
      f.write(f"{line}\n")
  return notags_reads

Have filtered out unmapped reads from sorted.bam into mapped.sam. All reads should be assigned a strand in this case

In [110]:
def check_seq(str):
    import re
    chars = set('ACTG')
    seq = set(str.upper())
    if seq.issubset(chars):
        return True
    else:
        return False

print(check_seq("ACTGGggG"))


True


In [159]:
my_reads = read_samfile("sorted_reads/mapped_test_5000_R1.sam")[1][0:10]
#for read in my_reads:   
#    print(read)
 #   fb = find_barcodes2(read)
 #   print(fb)
 #   tg = find_tags2(read, "ACTTATCAGCCAACCTGTTA", 2)
 #   print(tg)
    
filter_mapped_reads2("sorted_reads/mapped_test_5000_R1.sam")


Total number of mapped reads:  1180
number of good reads (with tag):  1105
number of bad reads (with no tag):  7
Number of reads with duplicate barcode/starts:  68


['A01968:63:H77VYDSX5:4:1101:5267:1752_1:N:0:AACGTGAT_BC:GGGGGGGG\t16\tNC_002945.3\t186581\t60\t119M32S\t*\t0\t0\tCGCTGTTCTACGCCGACGGCACCACTATGTTGTTCGGTGATGCGAAGAAATCGGTTACCGAAGTCTCCGAGGAACTCAAGGCGTTGTAGCGCGCGAGCGCTGGCTCAGACGGGCGGATAACAGGTTGGCTGATAGTCCTCGGTCTCTAGAC\tF:FFF,FFFFFFFFFFFFFFF:FFF::FFFFFFFF:FFFFFF,FFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFF:FFFFF:FFFFF:FFFFFFFFFFF:FFFFFF:\tNM:i:1\tMD:Z:55G63\tAS:i:114\tXS:i:0',
 'A01968:63:H77VYDSX5:4:1101:21766:2300_1:N:0:AACGTGAT_BC:TATTTCAT\t16\tNC_002945.3\t215396\t60\t119M32S\t*\t0\t0\tTGCAGCTCTTCGTGCTCATCGCCGCGGCCCACGACGTGCGCTGCGACGTGGCATCGAATTCGCCGTTCGTGTACGCCTACGGGTTCGCCGAGGACATCGACACCAGCCACGCCCTATACCCCGTGGGCTGATAAGTCCCCGGTCTCTAGAC\tF::FF:F,F,:FFFF:,F:F,F,FFF,F,:F:FFF:,FFFF:,,FFFF,FFFFFFFFFFFF::FF,:F:FFF,FF:F::FFF::FFFFFF,FFF,FFFF,FFFF:,FF:FFFF:F:,F,F,,FF,,FFFF:FFFFFFFF:FFFFFFFFFFF\tNM:i:3\tMD:Z:22G6A32A56\tAS:i:104\tXS:i:0',
 'A01968:63:H77VYDSX5:4:1101:11559:2002_1:N:0:AACGTGAT_BC:ACACCACT\t16\tNC_0

6 of these 7 no tag reads have different part of the transposon tag in them, but not the part that ends in GTTA

+  ACTTATCAGCCAACCTGTTA-gDNA
-  gDNA-TAACAGGTTGGCTGATAAGT
            gDNA?-GCTGATAAGTCCCCGGTCTCTAGAC
 

These have parts of transposon tag, but either >2 matches, or missing the terminal end of Himar1 transposon which includes the TA site.

View these on artemis. Artemis won't view .sam, so have to change to .bam

In [ ]:
conda activate dna
samtools view -b -o tag_filtered_mapped_test_5000_R1.bam tag_filtered_mapped_test_5000_R1.sam

This samtools view won't work with the filtered because the @SQ lines are removed when filtered for mapped reads.

Can use -h to reset output with header--edited snakemake file for filtering by mapped reads.



6. Quantify reads at each TA site (need list of coordinates of TA sites for bovis to make wig files). Need to make sure start position parsed from the cigar string is accurate

How do left-most positions from reads align with TA sites?



## View on artemis

Reverse strand mapped read with start position calculated as 39,859 (left-most position + aligned len from cigar string). This is actually one off from real start position which is one too far right (5' for reverse). REad actually begins at 39858. TA site on - strand starts at 39858. Positive read starts at 39857 (correct left-most position from cigar) which coincides with TA starting at 39857. These are same TA site and should be counted as 2 insertions for one TA site. The site is referenced by + strand T of the TA in the file of ta sites (bovis_TA_sites.txt) found with insertion_site_finder function. Therefore, reverse strand reads need to subtract one from + strand coordinate of start to line up with the correct TA site (which is really the 'A' of the TA site on the reverse compl strand)

![Forward and reverse mapped to same TA site](images/F_R_sameTAsite.png)


For forward strand:  start == TA site position (+ strand coordinate)
for reverse strand:  start - 1 == TA site position (based on + strand coordinate)

Instead of changing start position for reverse reads, will quantify strands separately, with TA sites adjusted for reverse orientation?

Also, when TA site inserted before T in genomic, alignment len can be off by one (should have soft-clipped one more base as genomic DNA seq has TTA which matches transposon tag end -GTTA), should there be some leeway if start is within 1 base of TA site position?

![soft-clip one early](images/softclip_error.png)

In [152]:


def motif_finder(seq, motif):
    """Finds location of motif (substring) in sequence.
    Input       seq         sequence
                motif       desired subsequence
    Output      locations   list of sequence locations of substrings (one-indexed)
    """
    import re
    locations = []
    p = re.compile('(?='+motif+')')       # use lookahead to find all overlapping matches
    matches = p.finditer(seq)
    for match in matches:
        start = match.span()
        locations.append(start[0] + 1)   # add one to change from zero-indexing to sequence position   


    return locations

#**********************************************************************************

def find_insertion_sites(seq):
    """Finds all the locations of TA insertion sites in sequence

    Seems you only want TA positions from one strand for counting number of sites,
    as they are same site on either strand. When quantifying, need to get reads that start 
    """
    from Bio.Seq import Seq
    
    seq_obj         = Seq(seq)
    rec_seq         = str(Seq.reverse_complement(seq_obj))
    fwd_positions = motif_finder(seq, "TA")
    rev_positions = motif_finder(rec_seq, "TA")
    
    return fwd_positions, rev_positions

#**********************************************************************************


def open_fasta(refseq):
    with open(refseq, 'r') as file:
        seq = ''
        for line in file:
            if line[0] != '>':
                seq += line.strip()
    return seq
    


Make list of TA sites in reference genome

In [161]:
bovis_fasta = "ref_seqs/Mbovis_AF2122-97.fasta"
bovis_seq   = open_fasta(bovis_fasta)
# to look at only first 30,000 nts of genome
bovis_30 = bovis_seq[0:30001]

res = motif_finder(bovis_seq, "TA")
print(len(res))
print(res[0:10])

with open('bovis_TA_sites.txt', 'w') as outfile:
    results = find_insertion_sites(bovis_seq)
    print(len(results[0]))
    #73465
    print(results[0][0:10])
    print(len(results[1]))
    #73465
    #these will be from 5' end of rc strand (-), wont use these
    print(results[1][0:10])
    #these should coincide with 5' end of + strand
    print(results[1][-10:])
    #for site in results:
     #   print(site, file=outfile)


    


73465
[60, 72, 102, 188, 246, 333, 360, 426, 448, 471]
73465
[60, 72, 102, 188, 246, 333, 360, 426, 448, 471]
73465
[6, 24, 53, 92, 111, 120, 135, 195, 255, 259]
[4345021, 4345044, 4345066, 4345132, 4345159, 4345246, 4345304, 4345390, 4345420, 4345432]


In [163]:
whole_set = [1,2,3,4,5,5,5,5,6,7,8,9,10]
print(len(whole_set))
unique_set = set(whole_set)
print(unique_set)
len(unique_set)

13
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


10

Use similar strategy to tpp_tools.py: read_counts(ref,sam,vars) template_counts(ref,sam,bcfile,vars) and increase_counts(pos,sites, strand)

1. use find_barcode_starts to get list of barcode/starts for each read. should be same length as number of reads
    - change filter function to only look for tranposon tags
    - new function to identify starts/strand/barcode for each read and make into list(which will be iterated through)
2. make dicts of ta-sites and read counts: ordered dictionary (default is ordered), new values will overwrite old values
    - analyse each strand separately
    - make TA:read dict: 
        - for each site key, append list barcodes that have starts within 1 nt of TA site
        - add up total reads for each site (F and R)
    - make TA:template dict:
        - for each site key, reduce barcode list to unique sites
        - add up total reads for each site (F and R)
3. Write wig files for TA sites using TA:template dict




In [ ]:
#make new dictionary with start positions and strand? or re-use barcode/start--add strand info?



New sequence if I don't want to map first?

1. quality control and trim illumina seqs
2. add barcode to fastq sequence name 
3. filter fastq reads for transposon tag--have to look in both directions (won't know which way will map--search for both?)
4. trim off transposon tag from reads--look for tag at both ends where there is short fragment (TACCACGACCA on other end) 
4. map reads
5. remove duplicates with same barcode/start position (could do this after assigning to TA site)
6. quantify reads at each ta site and generate .wig file